# PROJECT TITLE: FINDING THE MOST INTERESTING TWEETS AND FAN SENTIMENT DURING A GAME TELECAST IN REALTIME

# PART 6: SET TEAM TWEET REFERS TO:

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
dfEv = pd.read_csv('events_proc.csv')
dfEv.drop(['Unnamed: 0'], axis = 1, inplace = True)
dfEv

,Timestamp,ExtraMins,Team1,Team2,Player,Team,Event,MatchNo,EventID,EventHr,EventMnt,Time,Date
0,0,0,FRANCE,ARGENTINA,NaN,NaN,START,1,1101,17,0,17:00,30-06-2018
1,1,0,FRANCE,ARGENTINA,GIROUD,FRANCE,FOUL,1,1102,17,1,17:01,30-06-2018
2,3,0,FRANCE,ARGENTINA,POGBA,FRANCE,FOUL,1,1103,17,3,17:03,30-06-2018
3,3,0,FRANCE,ARGENTINA,MATUIDI,FRANCE,FOUL,1,1104,17,3,17:03,30-06-2018
4,8,0,FRANCE,ARGENTINA,MASCHERANO,ARGENTINA,FOUL,1,1105,17,8,17:08,30-06-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,90,1,FRANCE,CROATIA,VRSALJKO,CROATIA,FOUL,16,16230,19,34,19:34,15-07-2018
1462,90,2,FRANCE,CROATIA,VRSALJKO,CROATIA,FOUL,16,16231,19,35,19:35,15-07-2018
1463,90,3,FRANCE,CROATIA,VRSALJKO,CROATIA,YELLOW CARD,16,16232,19,36,19:36,15-07-2018
1464,90,4,FRANCE,CROATIA,FEKIR,FRANCE,FOUL,16,16233,19,37,19:37,15-07-2018


In [3]:
def set_team_from_tweet(fn):
    ################################################################
    # STEP 1: INITIALIZE ARRAY'S TO COLLECT EVENT DATA INSIGHTS:
    # A. PLAYER TEAM SET - SET OF ALL PLAYER AND TEAMS THEY BELONG TO
    # B. ABB SET - SET OF ALL ABB'S, TWEET CONTENT INCLUDES ABB'S
    # C. KEY WORD SET - SET OF PLAYER NAMES, TEAM NAMES, ABB'S, COMBINED ABB'S
    # D. TEAM ABB SET - RELATES ABB TO TEAM NAME
    # E. TEAM-TEAM ABB SET - REALTES COMBINED ABB TO BOTH TEAM NAMES
    # F. TEAM LIST

    # Key Word Array
    kw_arr = []
    # Keyword Set
    kw_set = []

    # Player Team Array
    pt_arr = []
    # Player Team Set
    pt_set = []

    # Abbreviation Array
    abb_arr = []
    # Abbreviation Set
    abb_set = []

    # Team Abbreviation Array
    ta_arr = []
    # Team Abbreviation Set
    ta_set = []

    # Team-Team Abbreviation Array
    tta_arr = []
    # Team-Team Abbreviation Set
    tta_set = []

    # Team List
    team_list = []
    # Team Set
    team_set = []

    #################################################################################
    # STEP 2: BUILD THE RESPECTIVE SETS

    dfEv.Player = dfEv.Player.fillna('')
    dfEv.Team = dfEv.Team.fillna('')

    for i in range(1466):
        if dfEv['Player'][i] != '':
            pt_arr.append([dfEv['Player'][i].lower(), dfEv['Team'][i].lower()])
            ta_arr.append([dfEv['Team'][i].lower(), dfEv['Team'][i][:3].lower()])
            team_list.append(dfEv['Team'][i].lower())
            kw_arr.append(dfEv['Player'][i].lower())
            kw_arr.append(dfEv['Team'][i].lower())

    for i in range(1466):
        a = dfEv['Team1'][i][:3].lower()
        b = dfEv['Team2'][i][:3].lower()
        c = a + b
        abb_arr.append([a, b, c])
        tta_arr.append([dfEv['Team1'][i].lower(), dfEv['Team2'][i].lower(), c])
        kw_arr.append(a)
        kw_arr.append(b)
        kw_arr.append(c)


    for i in pt_arr:
        if i not in pt_set:
            pt_set.append(i)

    for i in abb_arr:
        if i not in abb_set:
            abb_set.append(i)

    for i in kw_arr:
        if i not in kw_set:
            kw_set.append(i)

    for i in ta_arr:
        if i not in ta_set:
            ta_set.append(i)

    for i in tta_arr:
        if i not in tta_set:
            tta_set.append(i)

    for i in team_list:
        if i not in team_set:
            team_set.append(i)

    ################################################################################
    # STEP 3: COLLECT BAG OF WORDS FROM TWEET DATA, HASHTAG DATA, USER MENTIONED NAMES / IDS

    df = pd.read_csv(fn)
    df.drop(['Unnamed: 0', 'HTs', 'UMN', 'UMID'], axis = 1, inplace = True)

    df1 = df["Tweet_Extra"]
    # Tweet Exra Content Array - Contains HTs, UMN, UMID data
    te = []
    for i in df1:
        x = i.split('[')
        y = x[1].split(']')
        z = y[0].split("'")
        te.append(z)
    for i in te:
        i.remove('')
    for i in te:
        for j in i:
            if any(c.isalpha() for c in j):
                pass
            else:
                i.remove(j)

    df1 = df["Tweet_Proc"]
    # Tweet Content Array - Contains Tweet Data
    tc = []
    for i in df1:
        x = i.split('[')
        y = x[1].split(']')
        z = y[0].split("'")
        tc.append(z)
    for i in tc:
        i.remove('')
    for i in tc:
        for j in i:
            if any(c.isalpha() for c in j):
                pass
            else:
                i.remove(j)

    ############################################################################################################
    # STEP 4: FIND INTERSECTIONS OF KEYWORDS AND TWEETS TO DRAW INSIGHTS AS TO WHICH TEAM IS BEING REFERRED TO:

    # ARR1 - INTERSECTION OF KEYWORDS AND TWEET CONTENT
    arr1 = [] 
    set_A = set(kw_set)
    for i in range(len(tc)):
        set_B = set(tc[i])
        arr1.append(list(set_A.intersection(set_B)))

    # ARR2 - INTERSECTION OF KEYWORDS AND TWEET EXTRAS
    arr2 = []
    for i in range(len(te)):
        set_C = set(te[i])
        arr2.append(list(set_A.intersection(set_C)))

    ##############################################################################################
    # STEP 5: IF PLAYER NAME / ABB KEYWORDS ARE FOUND ADD TEAM NAME ALSO
    for i in arr1:
        for j in i:
            for k in pt_set:
                if k[0] == j:
                    i.append(k[1])

    for i in arr2:
        for j in i:
            for k in pt_set:
                if k[0] == j:
                    i.append(k[1])

    for i in arr1:
        for j in i:
            for k in ta_set:
                if k[1] == j:
                    i.append(k[0])

    for i in arr2:
        for j in i:
            for k in ta_set:
                if k[1] == j:
                    i.append(k[0])

    for i in arr1:
        for j in i:
            for k in tta_set:
                if k[2] == j:
                    i.append(k[0])
                    i.append(k[1])

    for i in arr2:
        for j in i:
            for k in tta_set:
                if k[2] == j:
                    i.append(k[0])
                    i.append(k[1])

    #############################################################################
    # STEP 6: COMBINE DATA REMOVE KEYWORDS OTHER THAN DERIVED TEAM NAMES

    # ARR3 - COMBINE ARR1 AND ARR2 CONTENT
    arr3 = []
    for i in range(len(arr1)):
        arr3.append(arr1[i] + arr2[i])

    # ARR4 - WILL CONTAIN ONLY TEAM REFERRED TO
    arr4 = []
    for i in arr3:
        tmp = []
        for j in i:
            if j in team_set:
                tmp.append(j)
        arr4.append(tmp)

    df['TeamsRef'] = arr4
    return df

In [4]:
df30 = set_team_from_tweet('tweet_3006.csv')
df30

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,Want to make this summer football that little ...,543,"['win', 'worldcup', 'bargain booze', 'bargainb...",30,6,2,56,"['want', 'make', 'summer', 'football', 'little...",[]
1,Didier Deschamps has confirmed Benjamin Mendy ...,23,"['fra', 'arg', 'city watch', 'city_watch']",30,6,2,56,"['didier', 'deschamps', 'ha', 'confirmed', 'be...","[argentina, france]"
2,Please play Power by EXO EXO also performed du...,1300,"['', 'aeri days', 'fifa world cup ?', 'aeriday...",30,6,2,56,"['please', 'play', 'power', 'exo', 'exo', 'als...",[]
3,weeks teams games goals The group stage was ev...,1259,"['worldcup', 'b/r football', 'brfootball']",30,6,2,56,"['week', 'team', 'game', 'goal', 'the', 'group...",[]
4,Please play Power by EXO EXO also performed du...,1300,"['', 'aeri days', 'fifa world cup ?', 'aeriday...",30,6,2,56,"['please', 'play', 'power', 'exo', 'exo', 'als...",[]
...,...,...,...,...,...,...,...,...,...
74995,Didier Drogba Paul Pogba carried the team as l...,420,"['worldcup', 'devils of united', 'devilsofunit...",30,6,20,26,"['didier', 'drogba', 'paul', 'pogba', 'carried...","[france, france]"
74996,Attention Incondicionales If you are looking f...,15,"['', 'mexican national team', 'miseleccionmxen']",30,6,20,26,"['attention', 'incondicionales', 'if', 'lookin...",[]
74997,can watch this shot on loop,489,"['fraarg', 'worldcup', 'mask ishan', 'mr_lolwa']",30,6,20,26,"['watch', 'shot', 'loop']","[france, argentina]"
74998,Best day of the so far coming up imo,67,"['worldcup', 'david preece', 'davidpreece12']",30,6,20,26,"['best', 'day', 'far', 'coming', 'imo']",[]


In [5]:
df01 = set_team_from_tweet('tweet_0107.csv')
df01

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,Uruguay erupts in ecstasy after eliminating Po...,39,"['worldcup', 'rt sport', 'rtsportnews']",1,7,2,24,"['uruguay', 'erupts', 'ecstasy', 'eliminating'...","[uruguay, portugal]"
1,Neither Cristiano Ronaldo or Lionel Messi have...,1719,"['', 'optajoe', 'optajoe']",1,7,2,24,"['neither', 'cristiano', 'ronaldo', 'lionel', ...","[portugal, argentina]"
2,French commentators reaction to Pavard goal is...,8541,"['fra', 'fraarg', 'worldcup', 'zach townsend',...",1,7,2,24,"['french', 'commentator', 'reaction', 'pavard'...","[france, france, france, argentina]"
3,the internet is undefeated,846,"['worldcup', 'goat', '???????? ?', 'fonmand']",1,7,2,24,"['internet', 'undefeated']",[]
4,BREAKING Javier Mascherano has confirmed his r...,2142,"['arg', 'squawka news', 'squawkanews']",1,7,2,24,"['breaking', 'javier', 'mascherano', 'ha', 'co...","[argentina, argentina]"
...,...,...,...,...,...,...,...,...,...
56862,Its special,0,"['worldcup', '', '']",1,7,23,59,"['its', 'special']",[]
56863,Only two goalkeepers in history have saved thr...,263,"['worldcup', 'por', 'eng', 'match of the day',...",1,7,23,59,"['only', 'two', 'goalkeeper', 'history', 'save...","[portugal, england, portugal]"
56864,Heartbreak for as Denmark exit the after penal...,423,"['worldcup', 'tottenham hotspur', 'christian e...",1,7,23,59,"['heartbreak', 'denmark', 'exit', 'penalty', '...","[croatia, denmark]"
56865,penalty missed CAn have your opinion of ME Wha...,1,"['modric', 'bbcworldcup', 'esp', 'congratulati...",1,7,23,59,"['penalty', 'missed', 'can', 'opinion', 'me', ...","[russia, croatia]"


In [6]:
df02 = set_team_from_tweet('tweet_0207.csv')
df02

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,Captain,85,"['worldcup', 'esp', '?? vs ?? football tweet',...",2,7,0,0,['captain'],[]
1,this England print signed by hat trick hero Si...,176,"['win', 'worldcup', 'competition', 'spirit of ...",2,7,0,0,"['england', 'print', 'signed', 'hat', 'trick',...",[england]
2,Is this coincidence,8412,"['worldcup', 'peter schmeichel', 'pschmeichel1']",2,7,0,0,"['is', 'coincidence']",[]
3,Best world cup eva,0,"['worldcup', 'whatatimetobealive', '', '']",2,7,0,0,"['best', 'world', 'cup', 'eva']",[]
4,Kasper Schmeichel takes the final award of the...,2240,"['manofthematch', 'croden', 'worldcup', 'fifa ...",2,7,0,0,"['kasper', 'schmeichel', 'take', 'final', 'awa...","[croatia, denmark]"
...,...,...,...,...,...,...,...,...,...
63128,It not the day David De Gea want to remember a...,5,"['', 'united indonesia', 'david de gea', 'utdi...",2,7,4,35,"['it', 'day', 'david', 'de', 'gea', 'want', 'r...",[]
63129,Got favourite tune Tell us below and your choi...,12385,"['worldcup', 'fifastadiumdj', 'fifa world cup ...",2,7,4,35,"['got', 'favourite', 'tune', 'tell', 'u', 'cho...",[]
63130,Pure drama Despite Kasper Schmeichels heroics ...,748,"['den', 'worldcup', 'premier league', 'premier...",2,7,4,35,"['pure', 'drama', 'despite', 'kasper', 'schmei...",[denmark]
63131,Japan have never made it past the last eight s...,219,"['', 'fifa world cup ?', '????????', 'fifaworl...",2,7,4,35,"['japan', 'never', 'made', 'past', 'last', 'ei...",[japan]


In [7]:
df04 = set_team_from_tweet('tweet_0407.csv')
df04

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,Seven from Bar into the quarter finals of the ...,297,"['worldcup', 'barçaworldcup', 'fc barcelona', ...",4,7,1,2,"['seven', 'bar', 'quarter', 'final', 'heres', ...",[]
1,couldn be prouder of victory in penalty shooto...,3559,"['', 'kensington palace', 'england', 'kensingt...",4,7,1,2,"['prouder', 'victory', 'penalty', 'shootout', ...",[england]
2,Get in there Proud Bottle Character,10156,"['threelions', 'worldcup', 'eng', 'harry kane'...",4,7,1,2,"['get', 'proud', 'bottle', 'character']",[england]
3,First World Cup penalty shootout win Sure we w...,110,"['eng', 'worldcup', 'copa90', 'copa90']",4,7,1,2,"['first', 'world', 'cup', 'penalty', 'shootout...",[england]
4,Get in there Proud Bottle Character,10156,"['threelions', 'worldcup', 'eng', 'harry kane'...",4,7,1,2,"['get', 'proud', 'bottle', 'character']",[england]
...,...,...,...,...,...,...,...,...,...
34995,Theres long way to go but if England win the t...,127,"['worldcup', 'jeremy corbyn', 'jeremycorbyn']",4,7,19,27,"['theres', 'long', 'way', 'go', 'england', 'wi...",[england]
34996,Even Putin knows,2788,"['eng', 'worldcup', 'sportsjoe', 'sportsjoe_uk']",4,7,19,27,"['even', 'putin', 'know']",[england]
34997,POWER by EXO To quote EXO not only is Power dy...,4657,"['', 'exo schedule', 'fifa world cup ?', 'exo_...",4,7,19,28,"['power', 'exo', 'to', 'quote', 'exo', 'power'...",[]
34998,worldcup is big football festival we need to e...,0,"['', 'fifa world cup ?', 'fifaworldcup']",4,7,19,28,"['worldcup', 'big', 'football', 'festival', 'n...",[]


In [8]:
df10 = set_team_from_tweet('tweet_1007.csv')
df10

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,WE ARE LIVE Schedule Build rollercoaster in wi...,66,"['fortnite', 'bel', 'fra', 'worldcup', 'nick',...",10,7,18,3,"['we', 'are', 'live', 'schedule', 'build', 'ro...","[france, belgium]"
1,Tonight an interesting one for Thierry Henry,166,"['worldcup', 'fra', 'bel', '?? vs ?? football ...",10,7,18,3,"['tonight', 'interesting', 'one', 'thierry', '...","[france, belgium]"
2,Good luck in the semi finals guys,710,"['worldcup', 'real madrid c.f. ????', 'raphaël...",10,7,18,3,"['good', 'luck', 'semi', 'final', 'guy']",[]
3,I ve got my pick to win the Let go,0,"['worldcup', 'england', 'gamefeels', '', '']",10,7,18,3,"['i', 'got', 'pick', 'win', 'let', 'go']",[england]
4,In Russia it illegal to display the LGBT pride...,11666,"['worldcup', 'jason ball', 'jasonballau']",10,7,18,3,"['in', 'russia', 'illegal', 'display', 'lgbt',...",[russia]
...,...,...,...,...,...,...,...,...,...
58869,Retweet for Eden Hazard Like for Mbappe,95,"['worldcup', 'frabel', 'azuwan shah', 'azuwanv...",10,7,23,59,"['retweet', 'eden', 'hazard', 'like', 'mbappe']","[belgium, france, france, belgium]"
58870,In Russia it illegal to display the LGBT pride...,17487,"['worldcup', 'jason ball', 'jasonballau']",10,7,23,59,"['in', 'russia', 'illegal', 'display', 'lgbt',...",[russia]
58871,When you are an assistant manager with Belgium...,358,"['frabel', 'worldcup', 'match of the day', 'bb...",10,7,23,59,"['when', 'assistant', 'manager', 'belgium', 'a...","[belgium, france, france, belgium]"
58872,Didier Deschamps as player captains at first f...,121,"['fra', 'worldcup', 'fra', 'squawka news', 'sq...",10,7,23,59,"['didier', 'deschamps', 'player', 'captain', '...",[france]


In [9]:
df11 = set_team_from_tweet('tweet_1107.csv')
df11

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,Belgium let Hazard down,0,"['worldcup', '', '']",11,7,0,0,"['belgium', 'let', 'hazard']","[belgium, belgium]"
1,This is what it means to reach final,1082,"['worldcup', 'manchester united', 'manutd']",11,7,0,0,"['this', 'mean', 'reach', 'final']",[]
2,FULL TIME France beat Belgium to enter their t...,43,"['worldcup', 'fra', 'bel', 'fifaworldcup2018',...",11,7,0,0,"['full', 'time', 'france', 'beat', 'belgium', ...","[belgium, france, france, belgium, france, bel..."
3,After yrs in London the Irish Elderly Advice N...,1,"['eng', 'worldcup', 'irish elderly advice netw...",11,7,0,0,"['after', 'yr', 'london', 'irish', 'elderly', ...",[england]
4,Producing News at Ten tonight President Higgin...,1,"['', ""gareth o'connor"", 'garethoconnor']",11,7,0,0,"['producing', 'news', 'ten', 'tonight', 'presi...",[]
...,...,...,...,...,...,...,...,...,...
46121,Lets do this,593,"['worldcup', 'engcro', 'manchester united', 'm...",11,7,22,49,['lets'],[]
46122,Issa draw Then wait for the drama in the extra...,8,"['worldcup', 'smasha boi ?', 'chusuuz']",11,7,22,49,"['issa', 'draw', 'then', 'wait', 'drama', 'ext...",[]
46123,If beats we ll giveaway World Cup Jersey Retwe...,100,"['croatia', 'england', 'modric', 'eng', 'true ...",11,7,22,49,"['if', 'beat', 'giveaway', 'world', 'cup', 'je...","[croatia, england, croatia, england]"
46124,Alexa knows it,1,"['vindaloo', 'engcro', 'worldcup', 'adam dowli...",11,7,22,49,"['alexa', 'know']",[]


In [10]:
df12 = set_team_from_tweet('tweet_1207.csv')
df12

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,This is what it looks like when you get mobbed...,780,"['worldcup', 'afp news agency', 'yuri cortez',...",12,7,0,20,"['this', 'look', 'like', 'get', 'mobbed', 'cel...",[]
1,Absolutely gutted that we lost tonight But d n...,2,"['worldcup', 'england', 'england']",12,7,0,20,"['absolutely', 'gutted', 'lost', 'tonight', 'b...",[england]
2,BREAKING NEWS UPDATE FROM SIR BRYN TERFEL,136,"['worldcup', 'theyrecominghome', 'welsh bolloc...",12,7,0,20,"['breaking', 'news', 'update', 'from', 'sir', ...",[]
3,If get beaten so will she increases by when En...,460,"['england', 'domesticviolence', 'shsc', 'shscft']",12,7,0,20,"['if', 'get', 'beaten', 'increase', 'england',...","[england, england]"
4,Croatia position since their first Worldcup in...,12,"['', 'iam_aderopo', 'ebenald25']",12,7,0,20,"['croatia', 'position', 'since', 'first', 'wor...",[croatia]
...,...,...,...,...,...,...,...,...,...
14995,Croatia Croatia Croatia,134,"['worldcup', 'worldcup2018', 'england', 'croat...",12,7,0,28,"['croatia', 'croatia', 'croatia']","[croatia, croatia, england]"
14996,Croatia is really good team and they finally s...,0,"['croeng', 'worldcup', 'equipe de france', 'eq...",12,7,0,28,"['croatia', 'really', 'good', 'team', 'finally...","[croatia, croatia, england]"
14997,These memories that will live on forever The T...,123,"['worldcup', 'budweiser uk', 'budweiseruk']",12,7,0,28,"['these', 'memory', 'live', 'forever', 'the', ...",[]
14998,You come back to England with your heads held ...,156,"['eng', 'talksport', 'mark saggers', 'talkspor...",12,7,0,28,"['you', 'come', 'back', 'england', 'head', 'he...","[england, england]"


In [11]:
df15 = set_team_from_tweet('tweet_1507.csv')
df15

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,ball can change the world Its not about the ba...,231,"['', 'sadhguru', 'sadhgurujv']",15,7,7,33,"['ball', 'change', 'world', 'its', 'ball', 'ki...",[]
1,This is it Final practice session next,200,"['worldcupfinal', 'flamingpride', 'croatia', '...",15,7,7,33,"['this', 'final', 'practice', 'session', 'next']",[croatia]
2,Cleopatra the tapir predicts Croatia to win final,67,"['worldcup', 'fifaworldcup', 'cgtn', 'cgtnoffi...",15,7,7,33,"['cleopatra', 'tapir', 'predicts', 'croatia', ...",[croatia]
3,vs Great defense by Alderweireld,702,"['belgium', 'england', 'worldcup', 'football g...",15,7,7,33,"['v', 'great', 'defense', 'alderweireld']","[belgium, england, belgium]"
4,The final Time to make history,1276,"['worldcup', 'b/r football', 'brfootball']",15,7,7,33,"['the', 'final', 'time', 'make', 'history']",[]
...,...,...,...,...,...,...,...,...,...
164995,Congratulations Pobro for making the Mancheste...,0,"['worldcupfinal', 'worldcup', '', '']",15,7,21,16,"['congratulations', 'pobro', 'making', 'manche...",[]
164996,Buyan the bear predicts Croatia will beat Fran...,254,"['', 'reuters top news', 'reuters']",15,7,21,16,"['buyan', 'bear', 'predicts', 'croatia', 'beat...","[croatia, france]"
164997,Dear France Congratulations on winning the of ...,21424,"['worldcup', 'khaled beydoun', 'khaledbeydoun']",15,7,21,16,"['dear', 'france', 'congratulations', 'winning...",[france]
164998,Congratulations to France the st African team ...,6019,"['worldcup', 'troll football', 'trollfootball']",15,7,21,16,"['congratulations', 'france', 'st', 'african',...",[france]


In [12]:
df16 = set_team_from_tweet('tweet_1607.csv')
df16

,Tweet,weight,Tweet_Extra,Day,Mon,Hr,Mnt,Tweet_Proc,TeamsRef
0,Just two UFC athletes enjoying the Final in Ru...,1400,"['worldcup', 'ufc', 'ufc']",16,7,1,49,"['just', 'two', 'ufc', 'athlete', 'enjoying', ...",[russia]
1,Croatian president Kolinda Grabar Kitarovi too...,705,"['', 'nicolas-patience basabose', 'mrbasabose']",16,7,1,49,"['croatian', 'president', 'kolinda', 'grabar',...",[]
2,FIFA Young Player Award Kylian MBAPPE,9290,"['fra', 'worldcup', 'fifa world cup', 'fifawor...",16,7,1,49,"['fifa', 'young', 'player', 'award', 'kylian',...","[england, france, france]"
3,VIDEO Power being played at the Fifa WorldCup ...,12286,"['exo', 'exo', 'exoglobal', 'exoglobal']",16,7,1,49,"['video', 'power', 'played', 'fifa', 'worldcup...",[]
4,Dear France Congratulations on winning the of ...,96731,"['worldcup', 'khaled beydoun', 'khaledbeydoun']",16,7,1,49,"['dear', 'france', 'congratulations', 'winning...",[france]
...,...,...,...,...,...,...,...,...,...
14995,Please guys please please help me ouuttt Pleas...,87,"['', 'shakazuluonthebeats', '#recognizemeep - ...",16,7,2,14,"['please', 'guy', 'please', 'please', 'help', ...",[]
14996,Dear France Congratulations on winning the of ...,96217,"['worldcup', 'khaled beydoun', 'khaledbeydoun']",16,7,2,14,"['dear', 'france', 'congratulations', 'winning...",[france]
14997,France have won the FIFA in Moscow,63116,"['fra', 'worldcup', 'fracro', 'worldcupfinal',...",16,7,2,14,"['france', 'fifa', 'moscow']","[france, france, france, croatia]"
14998,Islamophobia Xenophobia and racism are global ...,1,"['', 'alex florack', 'crimsonsith720']",16,7,2,14,"['islamophobia', 'xenophobia', 'racism', 'glob...",[]


In [13]:
rows = df30.shape[0]
rows += df01.shape[0]
rows += df02.shape[0]
rows += df04.shape[0]
rows += df10.shape[0]
rows += df11.shape[0]
rows += df12.shape[0]
rows += df15.shape[0]
rows += df16.shape[0]
rows

530000